In [18]:
pip install ultralytics

  Using cached ultralytics-8.3.78-py3-none-any.whl.metadata (35 kB)
  Using cached torch-2.6.0-cp312-cp312-win_amd64.whl.metadata (28 kB)
  Using cached torchvision-0.21.0-cp312-cp312-win_amd64.whl.metadata (6.3 kB)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
Using cached ultralytics-8.3.78-py3-none-any.whl (921 kB)
Using cached torch-2.6.0-cp312-cp312-win_amd64.whl (204.1 MB)
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
Using cached torchvision-0.21.0-cp312-cp312-win_amd64.whl (1.6 MB)
Using cached ultralytics_thop-2.0.14-py3-none-any.whl (26 kB)
  Attempting uninstall: sympy
    Found existing installation: sympy 1.12
    Uninstalling sympy-1.12:
      Successfully uninstalled sympy-1.12
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os
import shutil
import random
import sys
import cv2
from ultralytics import YOLO

In [2]:
data_dir = "C:/Users/Owen/Desktop/Yolo/data/Kunci gitar/"
train_dir = "C:/Users/Owen/Desktop/Yolo/data/images/train"
test_dir = "C:/Users/Owen/Desktop/Yolo/data/images/test/"

In [69]:
import os
import shutil
import random

# Direktori dataset
data_dir = "C:/Users/Owen/Desktop/Yolo/data/Kunci gitar"  # Sesuaikan path dengan folder gambar awal
labels_dir = "C:/Users/Owen/Desktop/Yolo/data/labels"  # Folder tempat semua label .txt tersimpan
train_dir = "C:/Users/Owen/Desktop/Yolo/data/images/train/"
test_dir = "C:/Users/Owen/Desktop/Yolo/data/images/test"
train_labels_dir = "C:/Users/Owen/Desktop/Yolo/data/labels/train"
test_labels_dir = "C:/Users/Owen/Desktop/Yolo/data/labels/test"

# Pastikan folder tujuan ada
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)
os.makedirs(train_labels_dir, exist_ok=True)
os.makedirs(test_labels_dir, exist_ok=True)

# Dapatkan semua gambar
all_images = [f for f in os.listdir(data_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]

# Acak urutan gambar
random.shuffle(all_images)

# Tentukan proporsi train-test (80%-20%)
split_index = int(0.8 * len(all_images))
train_images = all_images[:split_index]
test_images = all_images[split_index:]

# Fungsi untuk memindahkan gambar dan label yang sesuai
def move_files(image_list, target_img_dir, target_label_dir):
    for img in image_list:
        # Pindahkan gambar
        shutil.move(os.path.join(data_dir, img), os.path.join(target_img_dir, img))

        # Pindahkan label jika ada
        label_file = os.path.splitext(img)[0] + ".txt"  # Ubah ekstensi ke .txt
        label_path = os.path.join(labels_dir, label_file)

        if os.path.exists(label_path):  # Cek apakah file label ada
            shutil.move(label_path, os.path.join(target_label_dir, label_file))

# Pindahkan train set
move_files(train_images, train_dir, train_labels_dir)

# Pindahkan test set
move_files(test_images, test_dir, test_labels_dir)

print(f"Total Train Images: {len(train_images)}")
print(f"Total Test Images: {len(test_images)}")

Total Train Images: 323
Total Test Images: 81


In [71]:
print(f"Total Train Images: {len(train_images)}")
print(f"Total Test Images: {len(test_images)}")

Total Train Images: 323
Total Test Images: 81


In [3]:
min_thresh = 0.50                      # Minimum detection threshold
cam_index = 0                          # Index of USB camera
imgW, imgH = 1280, 720                 # Resolution to run USB camera at
record = False   

In [5]:
import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):
    # Cek apakah classes.txt ada
    if not os.path.exists(path_to_classes_txt):
        print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
        return
    
    # Baca file classes.txt untuk mendapatkan daftar nama kelas
    with open(path_to_classes_txt, 'r') as f:
        classes = [line.strip() for line in f.readlines() if line.strip()]
    
    number_of_classes = len(classes)

    # Buat dictionary konfigurasi YOLO
    data = {
        'path': 'C:/Users/Owen/Desktop/Yolo/data',  # Path utama dataset
        'train': 'C:/Users/Owen/Desktop/Yolo/data/images/train',  # Folder gambar train
        'val': 'C:/Users/Owen/Desktop/Yolo/data/images/test',  # Folder gambar test
        'nc': number_of_classes,  # Jumlah kelas
        'names': classes  # Nama kelas dari classes.txt
    }

    # Simpan dictionary ke dalam file YAML
    with open(path_to_data_yaml, 'w') as f:
        yaml.dump(data, f, sort_keys=False)
    
    print(f'Config file created at {path_to_data_yaml}')

# Jalankan fungsi untuk membuat data.yaml
path_to_classes_txt = "C:/Users/Owen/Desktop/Yolo/data/classes.txt"
path_to_data_yaml = "C:/Users/Owen/Desktop/Yolo/data/data.yaml"

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')
!cat /Users/Owen/Desktop/Yolo/data/data.yaml

Config file created at C:/Users/Owen/Desktop/Yolo/data/data.yaml

File contents:



'cat' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")


False
0
No GPU detected


In [9]:
label_path = "/Users/Owen/Desktop/Yolo/data/labels/train"

for file in os.listdir(label_path):
    if file.endswith(".txt"):
        with open(os.path.join(label_path, file), 'r') as f:
            lines = f.readlines()
            if not lines:
                print(f"⚠️ WARNING: Label kosong di {file}")
            for line in lines:
                parts = line.strip().split()
                if len(parts) != 5:
                    print(f"⚠️ ERROR: Format salah di {file}: {line}")
                try:
                    cls, x, y, w, h = map(float, parts)
                    if not (0 <= x <= 1 and 0 <= y <= 1 and 0 <= w <= 1 and 0 <= h <= 1):
                        print(f"⚠️ ERROR: Koordinat di luar batas di {file}: {line}")
                except ValueError:
                    print(f"⚠️ ERROR: Data tidak valid di {file}: {line}")


In [11]:
import os

label_path = "/Users/Owen/Desktop/Yolo/data/labels/train"
label_files = os.listdir(label_path)

if len(label_files) == 0:
    print("Label kosong! Pastikan ada file .txt di dalamnya.")
else:
    print(f"Ada {len(label_files)} label di dalam {label_path}")

Ada 323 label di dalam /Users/Owen/Desktop/Yolo/data/labels/train


In [ ]:
!yolo detect train data=/Users/Owen/Desktop/Yolo/data/data.yaml model=yolo11s.pt epochs=40 imgsz=480

New https://pypi.org/project/ultralytics/8.3.79 available ðŸ˜ƒ Update with 'pip install -U ultralytics'
Ultralytics 8.3.78 ðŸš€ Python-3.12.4 torch-2.6.0+cpu CPU (AMD Ryzen 5 5500U with Radeon Graphics)
engine\trainer: task=detect, mode=train, model=yolo11s.pt, data=/Users/Owen/Desktop/Yolo/data/data.yaml, epochs=40, time=None, patience=100, batch=16, imgsz=480, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train19, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, e


train: Scanning C:\Users\Owen\Desktop\Yolo\data\labels\train...:   0%|          | 0/323 [00:00<?, ?it/s]
train: Scanning C:\Users\Owen\Desktop\Yolo\data\labels\train... 26 images, 0 backgrounds, 0 corrupt:   8%|8         | 26/323 [00:00<00:01, 226.42it/s]
train: Scanning C:\Users\Owen\Desktop\Yolo\data\labels\train... 75 images, 0 backgrounds, 0 corrupt:  23%|##3       | 75/323 [00:00<00:00, 340.12it/s]
train: Scanning C:\Users\Owen\Desktop\Yolo\data\labels\train... 109 images, 0 backgrounds, 0 corrupt:  34%|###3      | 109/323 [00:02<00:07, 28.71it/s]
train: Scanning C:\Users\Owen\Desktop\Yolo\data\labels\train... 323 images, 0 backgrounds, 0 corrupt: 100%|##########| 323/323 [00:02<00:00, 110.66it/s]

val: Scanning C:\Users\Owen\Desktop\Yolo\data\labels\test...:   0%|          | 0/81 [00:00<?, ?it/s]
val: Scanning C:\Users\Owen\Desktop\Yolo\data\labels\test... 14 images, 0 backgrounds, 0 corrupt:  17%|#7        | 14/81 [00:00<00:00, 94.80it/s]
val: Scanning C:\Users\Owen\Desktop\Yol

In [15]:
import os

# Path ke folder dataset
image_folder = "C:/Users/Owen/Desktop/Yolo/data/images/train/"
label_folder = "C:/Users/Owen/Desktop/Yolo/data/labels/train/"

# Ambil daftar nama file
image_files = {f.split('.')[0] for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png', '.jpeg'))}
label_files = {f.split('.')[0] for f in os.listdir(label_folder) if f.endswith('.txt')}

# Cari gambar yang tidak punya label
missing_labels = image_files - label_files
missing_images = label_files - image_files

print(f"Gambar tanpa label: {missing_labels}")
print(f"Label tanpa gambar: {missing_images}")

Gambar tanpa label: set()
Label tanpa gambar: set()


In [23]:
!yolo detect predict model=/Users/Owen/runs/detect/train19/weights/best.pt source="C:/Users/Owen/Desktop/Yolo/data/images/test" save=True

Ultralytics 8.3.78 ðŸš€ Python-3.12.4 torch-2.6.0+cpu CPU (AMD Ryzen 5 5500U with Radeon Graphics)
YOLO11s summary (fused): 100 layers, 9,414,735 parameters, 0 gradients, 21.3 GFLOPs

image 1/81 C:\Users\Owen\Desktop\Yolo\data\images\test\02192287-WIN_20250221_10_01_13_Pro.jpg: 288x480 1 CHORD AMAJ7, 126.1ms
image 2/81 C:\Users\Owen\Desktop\Yolo\data\images\test\0258738e-WIN_20250221_12_30_42_Pro.jpg: 288x480 1 CHORD A, 102.9ms
image 3/81 C:\Users\Owen\Desktop\Yolo\data\images\test\04dbaf42-maxresdefault_2.jpg: 288x480 1 CHORD G, 81.0ms
image 4/81 C:\Users\Owen\Desktop\Yolo\data\images\test\07ec7ee3-WIN_20250221_09_58_01_Pro.jpg: 288x480 1 CHORD A, 93.5ms
image 5/81 C:\Users\Owen\Desktop\Yolo\data\images\test\146b3db8-WIN_20250221_12_33_04_Pro.jpg: 288x480 1 CHORD G, 88.2ms
image 6/81 C:\Users\Owen\Desktop\Yolo\data\images\test\17cd1e35-WIN_20250221_12_25_51_Pro.jpg: 288x480 1 CHORD C, 90.1ms
image 7/81 C:\Users\Owen\Desktop\Yolo\data\images\test\1847ef39-WIN_20250221_12_29_44_Pro.jpg:

In [33]:
import glob
from IPython.display import Image, display
for image_path in glob.glob(f'C:/Users/Owen/runs/detect/predict3.jpg')[:10]:
  display(Image(filename=image_path, height=400))
  print('\n')

In [41]:
!cp /Users/Owen/Desktop/Yolo/data/train19/weights/best.pt /Users/Owen/Desktop/Yolo/Guitar_detect/Guitar_detect.pt

'cp' is not recognized as an internal or external command,
operable program or batch file.


In [9]:
foto = "C:/Users/Owen/Desktop/Yolo/data/images/test/"
labels = "C:/Users/Owen/Desktop/Yolo/data/labels/test/"
import os

label_path = "/Users/Owen/Desktop/Yolo/data/labels/train"
label_files = os.listdir(label_path)
label_test = os.listdir(labels)
foto_test = os.listdir(foto)
foto_train = os.listdir(image_folder)

print(f"panjang dari train image", len(foto_train))
print(f"panjang dari label train", len(label_files))
print(f"dari test image", len(foto_test))
print(f"dari test label", len(label))

panjang dari train image 323
panjang dari label train 323
dari test image 81
dari test label 81
